## Step 1 - Configure the Venafi control plane to for a new Firefly instance. 

This task would usually be done using the UI, however it can be fully automated using the API. As a convenience, and to get you started using Firefly in the shortest possible time, we will use a utility called "elevate" to do this. This will run in a Docker container and we'll use Docker Compose to orchestrate it. However before we can do this you need will need your API key as described in the project documentation. 

To run the the click through the following interactive steps. 

#### 1. Edit the `.env` to include a valid Venafi Cloud API key if you've not already done this. 
#### 2. In the terminal, type or paste the following command: 

```docker compose --profile control-plane up```

This will automate the creation of the required configuration items in the Venafi control plane. You can take a look by clicking through the various Firefly configuration tabs in the Venafi Web UI. The following new items should now exist under each of the sections: 

* Teams : basic-demo
* Service Account: basic-demo
* Sub CA Providers: basic-demo
* Policies: basic-demo
* Configurations: basic-demo

For the purposes of this demonstrations some simple policies have been created. 

Lets start by adding our Venafi cloud API key to the environment. The following command creates a new `.env` file that contains an API key that will be used by the Docker compose file to generate a working Firefly configuration. Replace the placeholder text with your own API key. 

In [6]:
echo "TLSPC_API_KEY={PASTE API KEY HERE}" > .env

In [1]:
token=$( curl -d \
     -H "Content-Type: application/x-www-form-urlencoded" \
     -X POST http://localhost:8001/token \
     -k -s |  jq -r  '.access_token' )
echo VTOKEN=$token >> .env
echo ${token}
echo ${token} | jq -R 'split(".") | .[0],.[1] | @base64d | fromjson'



jq: error (at <stdin>:1): Malformed BOM (while parsing '��')


: 5

In [21]:
tmp=$(openssl req -nodes -newkey rsa:2048 -keyout example.key -out - -subj "/C=GB/ST=London/L=London/O=Global Security/OU=IT Department/CN=example.com" 2>/dev/null)
csr=$( jq -n --arg string "$tmp" '$string' | tr -d '"' )
echo $csr


-----BEGIN CERTIFICATE REQUEST-----\nMIICvDCCAaQCAQAwdzELMAkGA1UEBhMCR0IxDzANBgNVBAgMBkxvbmRvbjEPMA0G\nA1UEBwwGTG9uZG9uMRgwFgYDVQQKDA9HbG9iYWwgU2VjdXJpdHkxFjAUBgNVBAsM\nDUlUIERlcGFydG1lbnQxFDASBgNVBAMMC2V4YW1wbGUuY29tMIIBIjANBgkqhkiG\n9w0BAQEFAAOCAQ8AMIIBCgKCAQEAxY55O/8oTirfdOyBNCbYl6cxJQvNL/iSqG87\nQbYpdOycvQz3WcmIlk3et0/kxcQiquB3444yAZpKJlAB8oRPl5GWCRR9E4Sl38GO\nEE61au/g0zVdztcPwJi0fxCHEoaZt3F/4qd2Sc7tuFcyi3ne/uyuFhdlhQ0aM0Wm\nri3K419NI1a3hIbkffOqPe9MUkTVzkWlbat5t9ZbXE1ZF2T140NoYxx4RqEj1ctL\nAupcqzI1N0KAHsBrP+E6+6k6f3GtGHsKmQqYviyuRwu7QLrt09P8ORR1TTrNDsJS\nS/76y8ETUlCe56lBVNiviUjJsB6LmrYWmi67Cod3UXrKoFbnYQIDAQABoAAwDQYJ\nKoZIhvcNAQELBQADggEBAEA8YYOs66XEMf79NWj2mHQx7Bx/wH5ic1LnMzSGIpXl\nbEROmkrPEtQhVEZj1jdL7OQlxkdeomq49+nJmloHduOAw5RtkDutnIrJ3HaifmHF\nQ5QyToVh7lVWAwPsuvvherDdDUg+nZVwrZjBqsz5B30pqGCS0gxtv4SVXlHu0sgY\njo2qGCbDKzdYUTQvjRS29Rgf7BlN7ss4Q4rAEeCWVhcLJTtotyrIwNqt10mWbG+m\ndNWs9balRjaIuQfmc1K/s+lzkOdGKrPyUz+Jzc+bzyI8zgsZdSLdapmyOddTgQUj\ndKK8KKoDtTtj9tWr7ZjsmEnuHLVBp/9axT+F6sF

In [29]:
curl 'https://localhost:8289/v1/certificatesigningrequest' \
--header 'Content-Type: application/json' \
 -H "Authorization: Bearer $token" \
--data '{
    "request": "'"$csr"'",
    "policyName": "Basic Demo"
}' -k -s #| jq -r .certificateChain

{"error":"failed to issue certificate","error_description":"token authentication failed: failed to parse JWT: [token is unverifiable: error while executing keyfunc: the given key ID was not found in the JWKS]"}


In [ ]:
docker exec -it firefly-playground-nginx-1 docker-entrypoint.d/getcert.sh reload

In [ ]:
cat <<EOF | grpcurl -plaintext -d @ -unix /root/firefly/spirl/uds/grpc.sock certificates.service.v1alpha1.CertificateRequestService/Create
{
  "request": {
    "policy_name": "TLS Server EC Policy",
    "subject": {
      "common_name": "grpc-over-uds.vena.fi",
      "org_units": [
        "Product Management"
      ],
      "organization": "Venafi, Inc.",
      "locality": "Salt Lake City",
      "state": "Utah",
      "country": "US"
    },
    "alt_names": {
      "dns_names": [
        "grpc-over-uds.vena.fi"
      ],
      "uris": [
        "spiffe://vena.fi/ns/default/sa/workload"
      ]
    },
    "key_type": "EC_P256",
    "validity_period": "P3D"
  }
}
EOF